### Install Libraries

In [ ]:
%pip install pymupdf
%pip install openai
%pip install torch
%pip install transformers

Python interpreter will be restarted.
Python interpreter will be restarted.
Python interpreter will be restarted.
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.10.0.2
    Not uninstalling typing-extensions at /databricks/python3/lib/python3.9/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-2925a6e6-ea32-4267-a9e8-5d35574a906e
    Can't uninstall 'typing-extensions'. No files were found to uninstall.
  Attempting uninstall: distro
    Found existing installation: distro 1.4.0
    Not uninstalling distro at /usr/lib/python3/dist-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-2925a6e6-ea32-4267-a9e8-5d35574a906e
    Can't uninstall 'distro'. No files were found to uninstall.
Python interpreter will be restarted.
Python interpreter will be restarted.
Python interpreter will be restarted.
Python interpreter will be restarted.
Python interpreter will be restarted.


#### Functions to read pdf file and convert to text


In [ ]:
import fitz  # PyMuPDF
import os
from transformers import pipeline

pdf_dir = "../Team07_Assesment"

def pdf_to_text(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page_num in range(doc.page_count):
        page = doc.load_page(page_num)
        text += page.get_text()

    return text

pdf_texts = {}
for pdf_file in os.listdir(pdf_dir):
    if pdf_file.lower().endswith('.pdf'):
        pdf_path = os.path.join(pdf_dir, pdf_file)

        print(f"Extracting text from {pdf_path}")
        text = pdf_to_text(pdf_path)
        pdf_texts[pdf_file] = text

print(pdf_texts)

Extracting text from ../Team07_Assesment/Paris2024-QS-Athletics.pdf
{'Paris2024-QS-Athletics.pdf': ' EVENTS (48)\n QUOTAPLACES\n Adhere to the rules outlinedin World Athletics’Eligibility rules(Book C3.3).-\n Athletes who, whether directly or indirectly, are affiliated to, fall under the jurisdiction of or have otherwise agreed to abide by the rules and regulations of either the Russian Athletics Federation or the Belarus Athletic Federation or are citizens of and reside in Russia or Belarus are ineligible to qualifyto compete at the Olympic Games Paris 2024. C.\n ATHLETE ELIGIBILITY \n QUALIFICATION PATHWAY\n By achieving the entry standard (see section I.) within the respective qualification period outlined below. The entry standards, approved by the World Athletics Council in November 2022, have been determined in order to target the qualification of approximately 50% of the athletes. For the avoidance of doubt, any athlete achieving the entry standard will be deemed qualified, rega

#### Read pdf using Tesseract-OCR

In [ ]:
import fitz  # PyMuPDF
import pytesseract
from PIL import Image
import io
import os

# Function to extract text from a PDF file
def extract_text_from_pdf(pdf_path):
    text = ""
    pdf_document = fitz.open(pdf_path)
    pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files (x86)\Tesseract-OCR\tesseract.exe'

    # Loop through each page
    for page_num in range(len(pdf_document)):
        page = pdf_document.load_page(page_num)

        # Extract text
        text += page.get_text("text")

        # Extract images
        images = page.get_images(full=True)
        for image_index, img in enumerate(images):
            base_image = pdf_document.extract_image(img[0])
            image_bytes = base_image["image"]
            image_ext = base_image["ext"]
            image = Image.open(io.BytesIO(image_bytes))

            # Save the image temporarily
            image_path = f"image_{page_num+1}_{image_index+1}.{image_ext}"
            image.save(image_path)

            # Perform OCR on the image
            ocr_text = pytesseract.image_to_string(image)
            text += "\n\n[OCR Result from Image]:\n" + ocr_text

            # Optionally, remove the image after OCR processing
            os.remove(image_path)

    pdf_document.close()
    return text

# Provide the path to your PDF
pdf_path = "Paris2024-QS-Athletics.pdf"

# Extract text from PDF
extracted_text = extract_text_from_pdf(pdf_path)

# Save extracted text to a file
with open("extracted_text.txt", "w", encoding="utf-8") as text_file:
    text_file.write(extracted_text)

print("Text extraction completed.")


In [ ]:
def read_text_file(file_path):
    """Read the content of a text file and return it as a string."""
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.read()
        return content
    except Exception as e:
        return f"An error occurred: {str(e)}"

pdf_texts = read_text_file("../Team07_Assesment/team7_assessment.txt")

In [ ]:
pip install cohere

Python interpreter will be restarted.
Python interpreter will be restarted.


In [ ]:
import cohere

cohere_client = cohere.Client('key')
first_pdf_text = list(pdf_texts.values())[0]

def generate_text_with_cohere(question,context):
    prompt = f"Context: {first_pdf_text}\n\nQuestion: {question}\nAnswer:"
    try:
        response = cohere_client.generate(
            prompt=prompt,
            max_tokens=100,
            temperature=0.7
        )
        return response.generations[0].text.strip()
    except Exception as e:
        return f"An error occurred: {str(e)}"

questions = [
        "What is the qualification period for the marathon?",
        "What are the eligibility rules for athletes from Russia or Belarus?"
        ]
for question in questions:
    response = generate_text_with_cohere(question,first_pdf_text)
    print(f"question: {question}")
    print(f"Answer: {response}")
print(response)

question: What are the eligibility rules for athletes from Russia or Belarus?
Answer: I am unable to provide information on the eligibility rules for athletes from Russia or Belarus as the context you provided appears to be unrelated. Please provide a clear and relevant context for the information you are seeking and I will do my best to assist you.
I am unable to provide information on the eligibility rules for athletes from Russia or Belarus as the context you provided appears to be unrelated. Please provide a clear and relevant context for the information you are seeking and I will do my best to assist you.


#### Local Model : Hugging Face

In [ ]:
import transformers
import torch

def ask_local_model(question, context):
    model_id = "distilbert-base-cased-distilled-squad"

    pipeline = transformers.pipeline(
        "question-answering", model=model_id, tokenizer=model_id, device=0 if torch.cuda.is_available() else -1
    )
    result = pipeline(question=question, context=context)

    return result['answer']


In [ ]:
def main():
    first_pdf_text = list(pdf_texts.values())[0]
    questions = [
        "What is the qualification period for the marathon?",
        "What are the eligibility rules for athletes from Russia or Belarus?",
        "What are the entry standards for the marathon at the Paris 2024 Olympics?"
    ]
    for question in questions:
        local_answer = ask_local_model(question, first_pdf_text)
        print(f"question: {question}")
        print(f"Answer: {local_answer}")
main()

question: What is the qualification period for the marathon?
Answer: 31December 2022 –30June 2024
question: What are the eligibility rules for athletes from Russia or Belarus?
Answer: ineligible to qualifyto compete at the Olympic Games Paris 2024
question: What are the entry standards for the marathon at the Paris 2024 Olympics?
Answer: 50%
